# Préparation des données pour le classificateur de prunes africaines

Ce notebook présente le processus de préparation des données pour l'entraînement et l'évaluation du modèle de classification des prunes africaines. Nous allons explorer les données, appliquer des transformations et préparer les chargeurs de données (dataloaders) pour l'entraînement du modèle.

## Table des matières

1. [Configuration de l'environnement](#1-configuration-de-lenvironnement)
2. [Exploration des données](#2-exploration-des-données)
3. [Prétraitement des images](#3-prétraitement-des-images)
4. [Préparation des données pour le modèle à deux étapes](#4-préparation-des-données-pour-le-modèle-à-deux-étapes)
5. [Visualisation des données](#5-visualisation-des-données)
6. [Analyse de la distribution des classes](#6-analyse-de-la-distribution-des-classes)
7. [Conclusion](#7-conclusion)

## 1. Configuration de l'environnement

Commençons par importer les bibliothèques nécessaires et configurer notre environnement de travail.

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import glob
import random
import seaborn as sns

# Ajouter le répertoire parent au chemin pour pouvoir importer nos modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importer nos modules personnalisés
from data.data_preprocessing import (
    get_train_transforms, 
    get_val_transforms, 
    load_and_prepare_data,
    load_and_prepare_two_stage_data,
    visualize_batch,
    analyze_dataset_distribution
)

# Définir les chemins des données (à modifier selon votre configuration)
DATA_ROOT = "../data/raw"  # Chemin vers le répertoire de données brutes
PLUM_DATA_DIR = os.path.join(DATA_ROOT, "plums")  # Sous-dossier pour les prunes
NON_PLUM_DATA_DIR = os.path.join(DATA_ROOT, "non_plums")  # Sous-dossier pour les non-prunes

# Vérifier si les répertoires existent
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(PLUM_DATA_DIR, exist_ok=True)
os.makedirs(NON_PLUM_DATA_DIR, exist_ok=True)

# Définir les paramètres
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_WORKERS = 4
RANDOM_SEED = 42

# Fixer les seeds pour la reproductibilité
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

## 2. Exploration des données

Avant de commencer le prétraitement, explorons la structure des données pour comprendre comment elles sont organisées.

In [ ]:
def explore_data_directory(data_dir):
    """Explore un répertoire de données et affiche sa structure."""
    if not os.path.exists(data_dir):
        print(f"Le répertoire {data_dir} n'existe pas.")
        return
    
    print(f"Exploration du répertoire: {data_dir}")
    
    # Lister les sous-dossiers (classes)
    subdirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    print(f"Sous-dossiers trouvés: {subdirs}")
    
    # Compter les images par classe
    class_counts = {}
    for subdir in subdirs:
        subdir_path = os.path.join(data_dir, subdir)
        image_files = [f for f in os.listdir(subdir_path) 
                      if os.path.isfile(os.path.join(subdir_path, f)) and 
                      f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        class_counts[subdir] = len(image_files)
    
    # Afficher les résultats
    for cls, count in class_counts.items():
        print(f"  - {cls}: {count} images")
    
    # Afficher quelques exemples d'images si disponibles
    if subdirs:
        sample_class = subdirs[0]
        sample_dir = os.path.join(data_dir, sample_class)
        image_files = [f for f in os.listdir(sample_dir) 
                      if os.path.isfile(os.path.join(sample_dir, f)) and 
                      f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        
        if image_files:
            print(f"\nExemples d'images de la classe '{sample_class}':")
            for i, img_file in enumerate(image_files[:5]):  # Afficher jusqu'à 5 exemples
                print(f"  - {img_file}")
            
            # Afficher une image d'exemple
            if len(image_files) > 0:
                sample_img_path = os.path.join(sample_dir, image_files[0])
                try:
                    img = Image.open(sample_img_path)
                    plt.figure(figsize=(4, 4))
                    plt.imshow(img)
                    plt.title(f"Exemple d'image: {sample_class}/{image_files[0]}")
                    plt.axis('off')
                    plt.show()
                    
                    # Afficher les dimensions de l'image
                    print(f"Dimensions de l'image: {img.size}")
                except Exception as e:
                    print(f"Erreur lors de l'affichage de l'image: {e}")

# Explorer les répertoires de données
print("=== Exploration des données de prunes ===")
explore_data_directory(PLUM_DATA_DIR)

print("\n=== Exploration des données non-prunes ===")
explore_data_directory(NON_PLUM_DATA_DIR)

### Création de données d'exemple si nécessaire

Si vous n'avez pas encore de données, vous pouvez utiliser le code ci-dessous pour créer une structure de données d'exemple. Cela vous permettra de tester le notebook avant d'utiliser vos propres données.

In [ ]:
def create_sample_data(force_create=False):
    """Crée des données d'exemple pour tester le notebook."""
    # Vérifier si des données existent déjà
    plum_classes = [d for d in os.listdir(PLUM_DATA_DIR) if os.path.isdir(os.path.join(PLUM_DATA_DIR, d))]
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    
    if plum_classes and os.path.exists(non_plum_dir) and not force_create:
        print("Des données existent déjà. Utilisez force_create=True pour les remplacer.")
        return
    
    # Créer la structure de dossiers
    plum_classes = ["ripe", "unripe", "damaged", "diseased", "overripe", "healthy"]
    for cls in plum_classes:
        os.makedirs(os.path.join(PLUM_DATA_DIR, cls), exist_ok=True)
    
    os.makedirs(non_plum_dir, exist_ok=True)
    
    # Créer des images d'exemple (carrés colorés)
    colors = {
        "ripe": (150, 0, 0),      # Rouge foncé
        "unripe": (0, 150, 0),    # Vert foncé
        "damaged": (150, 100, 0), # Marron
        "diseased": (100, 0, 100),# Violet
        "overripe": (100, 0, 0),  # Rouge très foncé
        "healthy": (150, 50, 50)  # Rouge-rose
    }
    
    # Générer des images pour chaque classe de prune
    for cls, base_color in colors.items():
        for i in range(10):  # 10 images par classe
            # Ajouter un peu de variation aléatoire à la couleur
            color_var = [max(0, min(255, c + random.randint(-20, 20))) for c in base_color]
            
            # Créer une image
            img = Image.new('RGB', (224, 224), (255, 255, 255))
            pixels = img.load()
            
            # Dessiner un cercle approximatif avec la couleur
            center_x, center_y = 112, 112
            radius = 100 + random.randint(-10, 10)
            
            for x in range(img.width):
                for y in range(img.height):
                    dist = ((x - center_x) ** 2 + (y - center_y) ** 2) ** 0.5
                    if dist <= radius:
                        # Ajouter du bruit à chaque pixel
                        pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color_var]
                        pixels[x, y] = tuple(pixel_color)
            
            # Sauvegarder l'image
            img_path = os.path.join(PLUM_DATA_DIR, cls, f"{cls}_{i+1}.jpg")
            img.save(img_path)
    
    # Générer des images pour la classe non-prune
    for i in range(20):  # 20 images non-prune
        # Couleur aléatoire qui n'est pas proche des couleurs de prune
        color = (random.randint(0, 100), random.randint(150, 255), random.randint(150, 255))
        
        # Créer une image
        img = Image.new('RGB', (224, 224), (255, 255, 255))
        pixels = img.load()
        
        # Dessiner une forme aléatoire (carré ou triangle)
        shape = random.choice(['square', 'triangle'])
        
        if shape == 'square':
            # Dessiner un carré
            size = random.randint(100, 150)
            top_left = (random.randint(0, 224-size), random.randint(0, 224-size))
            
            for x in range(top_left[0], top_left[0] + size):
                for y in range(top_left[1], top_left[1] + size):
                    if 0 <= x < 224 and 0 <= y < 224:
                        # Ajouter du bruit à chaque pixel
                        pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                        pixels[x, y] = tuple(pixel_color)
        else:
            # Dessiner un triangle
            p1 = (random.randint(50, 174), random.randint(50, 174))
            p2 = (p1[0] + random.randint(30, 50), p1[1] + random.randint(30, 50))
            p3 = (p1[0] - random.randint(0, 30), p2[1])
            
            # Remplir le triangle (algorithme simple)
            min_x = min(p1[0], p2[0], p3[0])
            max_x = max(p1[0], p2[0], p3[0])
            min_y = min(p1[1], p2[1], p3[1])
            max_y = max(p1[1], p2[1], p3[1])
            
            for x in range(min_x, max_x + 1):
                for y in range(min_y, max_y + 1):
                    if 0 <= x < 224 and 0 <= y < 224:
                        # Vérification simple si le point est dans le triangle
                        if (x >= p1[0] and y >= p1[1] and x <= p2[0] and y <= p2[1]):
                            # Ajouter du bruit à chaque pixel
                            pixel_color = [max(0, min(255, c + random.randint(-10, 10))) for c in color]
                            pixels[x, y] = tuple(pixel_color)
        
        # Sauvegarder l'image
        img_path = os.path.join(non_plum_dir, f"non_plum_{i+1}.jpg")
        img.save(img_path)
    
    print(f"Données d'exemple créées avec succès!")
    print(f"- {len(plum_classes)} classes de prunes avec 10 images chacune")
    print(f"- 20 images non-prune")

# Créer des données d'exemple si nécessaire
# Décommentez la ligne suivante pour créer des données d'exemple
# create_sample_data(force_create=False)

## 3. Prétraitement des images

Examinons les transformations que nous allons appliquer aux images pour l'entraînement et la validation/test.

In [ ]:
# Afficher les transformations d'entraînement
train_transforms = get_train_transforms(img_size=IMG_SIZE)
print("Transformations pour l'entraînement:")
print(train_transforms)

# Afficher les transformations de validation/test
val_transforms = get_val_transforms(img_size=IMG_SIZE)
print("\nTransformations pour la validation/test:")
print(val_transforms)

### Visualisation des transformations

Visualisons l'effet des transformations sur quelques images d'exemple.

In [ ]:
def visualize_transformations(image_path, transforms_list, num_samples=5):
    """Visualise l'effet des transformations sur une image."""
    if not os.path.exists(image_path):
        print(f"L'image {image_path} n'existe pas.")
        return
    
    # Charger l'image
    original_img = Image.open(image_path).convert('RGB')
    
    # Afficher l'image originale et ses transformations
    plt.figure(figsize=(12, 4))
    
    # Image originale
    plt.subplot(1, num_samples+1, 1)
    plt.imshow(original_img)
    plt.title("Original")
    plt.axis('off')
    
    # Images transformées
    for i in range(num_samples):
        # Appliquer les transformations
        transformed_img = transforms_list(original_img)
        
        # Convertir le tenseur en image pour l'affichage
        if isinstance(transformed_img, torch.Tensor):
            # Dénormaliser
            mean = torch.tensor([0.485, 0.456, 0.406]).reshape(3, 1, 1)
            std = torch.tensor([0.229, 0.224, 0.225]).reshape(3, 1, 1)
            transformed_img = transformed_img * std + mean
            
            # Convertir en numpy et transposer
            transformed_img = transformed_img.numpy().transpose((1, 2, 0))
            
            # Clip pour s'assurer que les valeurs sont entre 0 et 1
            transformed_img = np.clip(transformed_img, 0, 1)
        
        # Afficher l'image transformée
        plt.subplot(1, num_samples+1, i+2)
        plt.imshow(transformed_img)
        plt.title(f"Transform {i+1}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Trouver une image d'exemple
def find_sample_image():
    """Trouve une image d'exemple dans les données."""
    # Chercher dans le répertoire des prunes
    for cls in os.listdir(PLUM_DATA_DIR):
        cls_dir = os.path.join(PLUM_DATA_DIR, cls)
        if os.path.isdir(cls_dir):
            for img_file in os.listdir(cls_dir):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    return os.path.join(cls_dir, img_file)
    
    # Si aucune image n'est trouvée dans les prunes, chercher dans les non-prunes
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    if os.path.exists(non_plum_dir):
        for img_file in os.listdir(non_plum_dir):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                return os.path.join(non_plum_dir, img_file)
    
    return None

# Visualiser les transformations sur une image d'exemple
sample_image = find_sample_image()
if sample_image:
    print(f"Visualisation des transformations d'entraînement sur {os.path.basename(sample_image)}:")
    visualize_transformations(sample_image, train_transforms)
    
    print(f"\nVisualisation des transformations de validation sur {os.path.basename(sample_image)}:")
    visualize_transformations(sample_image, val_transforms)
else:
    print("Aucune image d'exemple trouvée. Veuillez d'abord créer des données d'exemple ou ajouter vos propres images.")

## 4. Préparation des données pour le modèle à deux étapes

Notre modèle utilise une approche en deux étapes :
1. **Détection** : Déterminer si l'image contient une prune ou non
2. **Classification** : Si une prune est détectée, classifier son état

Préparons les données pour ces deux étapes.

In [ ]:
# Vérifier si les répertoires de données existent et contiennent des images
def check_data_availability():
    """Vérifie si les données sont disponibles pour l'entraînement."""
    # Vérifier le répertoire des prunes
    plum_classes = [d for d in os.listdir(PLUM_DATA_DIR) if os.path.isdir(os.path.join(PLUM_DATA_DIR, d))]
    if not plum_classes:
        print(f"Aucune classe de prune trouvée dans {PLUM_DATA_DIR}. Veuillez ajouter des données ou créer des exemples.")
        return False
    
    # Vérifier s'il y a des images dans chaque classe
    for cls in plum_classes:
        cls_dir = os.path.join(PLUM_DATA_DIR, cls)
        images = [f for f in os.listdir(cls_dir) 
                 if os.path.isfile(os.path.join(cls_dir, f)) and 
                 f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
        if not images:
            print(f"Aucune image trouvée dans la classe {cls}. Veuillez ajouter des données.")
            return False
    
    # Vérifier le répertoire des non-prunes
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    if not os.path.exists(non_plum_dir):
        print(f"Le répertoire {non_plum_dir} n'existe pas. Veuillez créer ce répertoire et y ajouter des images.")
        return False
    
    # Vérifier s'il y a des images dans le répertoire non-prune
    non_plum_images = [f for f in os.listdir(non_plum_dir) 
                      if os.path.isfile(os.path.join(non_plum_dir, f)) and 
                      f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    if not non_plum_images:
        print(f"Aucune image trouvée dans {non_plum_dir}. Veuillez ajouter des données.")
        return False
    
    return True

# Vérifier la disponibilité des données
data_available = check_data_availability()

if data_available:
    try:
        # Charger et préparer les données pour les deux étapes
        print("Chargement des données pour les deux étapes...")
        (detection_train_loader, detection_val_loader, detection_test_loader, detection_class_names), \
        (classification_train_loader, classification_val_loader, classification_test_loader, classification_class_names) = \
            load_and_prepare_two_stage_data(
                PLUM_DATA_DIR, 
                NON_PLUM_DATA_DIR,
                batch_size=BATCH_SIZE, 
                img_size=IMG_SIZE,
                num_workers=NUM_WORKERS
            )
        
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
        
        # Afficher les tailles des datasets
        print(f"\nTailles des datasets de détection:")
        print(f"  - Entraînement: {len(detection_train_loader.dataset)} images")
        print(f"  - Validation: {len(detection_val_loader.dataset)} images")
        print(f"  - Test: {len(detection_test_loader.dataset)} images")
        
        print(f"\nTailles des datasets de classification:")
        print(f"  - Entraînement: {len(classification_train_loader.dataset)} images")
        print(f"  - Validation: {len(classification_val_loader.dataset)} images")
        print(f"  - Test: {len(classification_test_loader.dataset)} images")
    except Exception as e:
        print(f"Erreur lors du chargement des données: {e}")
else:
    print("Veuillez d'abord créer des données d'exemple ou ajouter vos propres images.")

## 5. Visualisation des données

Visualisons quelques exemples d'images de chaque dataset pour mieux comprendre nos données.

In [ ]:
# Visualiser un batch d'images pour chaque étape
if data_available and 'detection_train_loader' in locals() and 'classification_train_loader' in locals():
    try:
        print("Visualisation d'un batch d'images pour la détection...")
        detection_batch_viz = visualize_batch(detection_train_loader, detection_class_names)
        
        print("\nVisualisation d'un batch d'images pour la classification...")
        classification_batch_viz = visualize_batch(classification_train_loader, classification_class_names)
        
        # Afficher les images sauvegardées
        if os.path.exists('batch_visualization.png'):
            plt.figure(figsize=(12, 10))
            img = plt.imread('batch_visualization.png')
            plt.imshow(img)
            plt.axis('off')
            plt.title('Visualisation du batch')
            plt.show()
    except Exception as e:
        print(f"Erreur lors de la visualisation des batches: {e}")

## 6. Analyse de la distribution des classes

Analysons la distribution des classes dans notre dataset pour vérifier si nous avons un équilibre entre les différentes classes.

In [ ]:
# Analyser la distribution des classes
if data_available:
    try:
        print("Analyse de la distribution des classes de prunes...")
        class_counts, distribution_img = analyze_dataset_distribution(PLUM_DATA_DIR)
        
        # Afficher les résultats
        print("\nDistribution des classes de prunes:")
        for cls, count in class_counts.items():
            print(f"  - {cls}: {count} images")
        
        # Afficher le graphique de distribution
        if os.path.exists('class_distribution.png'):
            plt.figure(figsize=(10, 6))
            img = plt.imread('class_distribution.png')
            plt.imshow(img)
            plt.axis('off')
            plt.title('Distribution des classes')
            plt.show()
            
        # Analyser l'équilibre des classes
        if class_counts:
            min_count = min(class_counts.values())
            max_count = max(class_counts.values())
            avg_count = sum(class_counts.values()) / len(class_counts)
            
            print(f"\nStatistiques de la distribution:")
            print(f"  - Nombre minimum d'images par classe: {min_count}")
            print(f"  - Nombre maximum d'images par classe: {max_count}")
            print(f"  - Nombre moyen d'images par classe: {avg_count:.1f}")
            
            # Vérifier le déséquilibre
            if max_count > 2 * min_count:
                print("\n⚠️ Attention: Le dataset présente un déséquilibre significatif entre les classes.")
                print("   Considérez l'utilisation de techniques comme la pondération des classes ou l'augmentation de données ciblée.")
            else:
                print("\n✅ Le dataset semble relativement équilibré entre les différentes classes.")
    except Exception as e:
        print(f"Erreur lors de l'analyse de la distribution des classes: {e}")

## 7. Conclusion

Dans ce notebook, nous avons exploré et préparé les données pour notre modèle de classification des prunes africaines. Nous avons :

1. Configuré notre environnement de travail
2. Exploré la structure des données
3. Examiné les transformations appliquées aux images
4. Préparé les données pour notre modèle à deux étapes (détection et classification)
5. Visualisé des exemples d'images de chaque dataset
6. Analysé la distribution des classes

Ces étapes sont essentielles pour comprendre nos données et préparer un modèle performant. Dans le prochain notebook, nous utiliserons ces données préparées pour entraîner notre modèle de classification.

### Prochaines étapes

- Entraînement du modèle de détection (prune vs non-prune)
- Entraînement du modèle de classification des prunes
- Évaluation des performances des modèles
- Optimisation des hyperparamètres
- Déploiement du modèle final